In [ ]:
from data_structures import PatientData
from sklearn.base import BaseEstimator
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from typing import Tuple
from sklearn.model_selection import train_test_split
import numpy as np
from dataclasses import dataclass
from typing import Dict, List
from sklearn.metrics import accuracy_score, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
from decoders import DecodingResult, ConceptDecoder, ConceptPairDataset, DecodingResultsManager

todo: imp dropout - add firing rate filter method to patient data to return neurons above threshold, implement by area stuff

In [ ]:
p566 = PatientData(pid='566')
p563 = PatientData(pid='563')
p562 = PatientData(pid='562')

In [ ]:
p566_mtl_filtered_neurons = p566.filter_mtl_neurons(p566.filter_neurons_by_fr((p566.times_dict['movie_start_rel'], p566.times_dict['preSleep_recall_start_rel']), threshold=0.1))
p563_mtl_filtered_neurons = p563.filter_neurons_by_fr((p563.times_dict['movie_start_rel'], p563.times_dict['preSleep_recall_start_rel']), threshold=0.1)
p562_mtl_filtered_neurons = p562.filter_neurons_by_fr((p562.times_dict['movie_start_rel'], p562.times_dict['preSleep_recall_start_rel']), threshold=0.1)


In [ ]:
concept_pairs_to_decode = []
concept_list = list(p566.movie_df.columns)
#remove non-concept columns if they exist
concept_list = [col for col in concept_list if col not in ['Frame', 'time_sec', 'rel_corrected_time_sec', 'uncorrected_time_sec']]


# doing to 5 for time

for i, concept1 in enumerate(concept_list[:]):
    for concept2 in concept_list[i+1:]: #avoid duplicates and self-pairs
        concept_pairs_to_decode.append((concept1, concept2))

print(f"Number of concept pairs to decode: {len(concept_pairs_to_decode)}")
print(concept_pairs_to_decode[:5]) # Print first 5 pairs as example

In [ ]:
pseudo_manager562 = DecodingResultsManager(
    patient_data=p562,
    concept_pairs=concept_pairs_to_decode,
    epoch='movie',
    standardize=False,
    pseudo=True,
    neurons=p562_filtered_neurons
    # no kwargs -> default
)

normal_manager562 = DecodingResultsManager(
    patient_data=p562,
    concept_pairs=concept_pairs_to_decode,
    epoch='movie',
    standardize=False,
    pseudo=False,
    neurons=p562_filtered_neurons)

In [ ]:
pseudo_manager563 = DecodingResultsManager(
    patient_data=p563,
    concept_pairs=concept_pairs_to_decode,
    epoch='movie',
    standardize=False,
    pseudo=True,
    neurons=p563_filtered_neurons
    # no kwargs -> default
)

normal_manager563 = DecodingResultsManager(
    patient_data=p563,
    concept_pairs=concept_pairs_to_decode,
    epoch='movie',
    standardize=False,
    pseudo=False,
    neurons=p563_filtered_neurons)

In [ ]:
pseudo_manager566 = DecodingResultsManager(
    patient_data=p566,
    concept_pairs=concept_pairs_to_decode,
    epoch='movie',
    standardize=False,
    pseudo=True,
    neurons=p566_filtered_neurons
    # no kwargs -> default
)

normal_manager566 = DecodingResultsManager(
    patient_data=p566,
    concept_pairs=concept_pairs_to_decode,
    epoch='movie',
    standardize=False,
    pseudo=False,
    neurons=p566_filtered_neurons)

todo: 
mtl neurons only - add filtering function
combine patients
8 best concepts

In [ ]:
for title, manager in {'normal_manager566': normal_manager566, 'normal_manager563': normal_manager563, 'normal_manager562': normal_manager562,
                       'pseudo_manager566': pseudo_manager566, 'pseudo_manager563': pseudo_manager563, 'pseudo_manager562': pseudo_manager562}.items():
    manager.run_decoding_for_pairs(num_iter=5)
    manager.plot_train_test_performance_heatmap(metric='test_accuracy')
    plt.savefig(f"{title}_test_accuracy")
    manager.plot_train_test_performance_heatmap() # auc roc
    plt.savefig(f"{title}_roc_auc")

